# install moabb & braindecode

In [ ]:
pip install moabb

In [ ]:
pip install braindecode

In [ ]:
pip install -U https://api.github.com/repos/shivar2/braindecode/zipball/master

# connect to drive

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import os, sys
os.chdir('/content/drive/MyDrive/MotorImagery-Classification')

# preproccess

## set MNE config

In [ ]:
import mne

mne.set_config('MNE_DATASETS_SCHIRRMEISTER2017_PATH', 'mne_data')

print(mne.utils.get_config())

## download or load dataset

In [ ]:
from braindecode.datasets.moabb import MOABBDataset
subject_id = 2
dataset = MOABBDataset(dataset_name="Schirrmeister2017", subject_ids=[subject_id])

# preproccess

In [9]:
from braindecode.datautil.preprocess import (
    exponential_moving_standardize, preprocess, Preprocessor)

from braindecode.datautil.windowers import create_fixed_length_windows
from braindecode.datautil.serialization import (
    save_concat_dataset, load_concat_dataset)

mne.set_log_level('ERROR')  # avoid messages everytime a window is extracted


In [ ]:
MNEPreproc(fn='resample', sfreq=10)

In [10]:
sfreq = 1000.0

In [11]:
n_times = dataset.datasets[0].raw.n_times #train-raw
n_times

3131420

In [12]:
n_times = dataset.datasets[1].raw.n_times #test-raw
n_times

614290

In [13]:
def custom_crop(raw, tmin=0.0, tmax=None, include_tmax=True):
    # crop recordings to tmin – tmax. can be incomplete if recording
    # has lower duration than tmax
    # by default mne fails if tmax is bigger than duration
    tmax = min((n_times - 1) /sfreq, tmax)
    raw.crop(tmin=tmin, tmax=tmax, include_tmax=include_tmax)

tmin = 1 * 60
tmax = 6 * 60

In [14]:
low_cut_hz = 4.  # low cut frequency for filtering
high_cut_hz = 38.  # high cut frequency for filtering
# Parameters for exponential moving standardization
factor_new = 1e-3
init_block_size = 1000

preprocessors = [ 
    Preprocessor(custom_crop, tmin=tmin, tmax=tmax, include_tmax=False,
                 apply_on_array=False),
    Preprocessor('pick_types', eeg=True, meg=False, stim=False),  # Keep EEG sensors
    Preprocessor(lambda x: x * 1e6),  # Convert from V to uV
    Preprocessor('filter', l_freq=low_cut_hz, h_freq=high_cut_hz),  # Bandpass filter
    Preprocessor(exponential_moving_standardize,  # Exponential moving standardization
                 factor_new=factor_new, init_block_size=init_block_size)
]

# Transform the data

preprocess(dataset, preprocessors)


In [15]:
window_size_samples = 1000
window_stride_samples = 1000
create_compute_windows = True

out_i = 0
errors = []
OUT_PATH = './tuh_sample/'
tuh_splits = dataset.split('run')
for rec_i, tuh_subset in tuh_splits.items():
      pass

## save sets

In [ ]:
import torch

torch.save(dataset, 'data-file/Schirrmeister2017/' + str(subject_id) + '.fif')